In [3]:
import pandas as pd

In [ ]:

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# from 2020-01-01 to 2020-11-30 
date_range = list(pd.bdate_range('20151201', '20201130',freq='1D').strftime("%Y-%m-%d"))
data_store = pd.DataFrame()
for date in date_range:
    url = 'https://www.wunderground.com/history/daily/KLGA/date/{}'.format(date)
    driver = webdriver.Chrome()
    driver.get(url)
    tables = WebDriverWait(driver,20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "mat-table.cdk-table.mat-sort.ng-star-inserted")))
    for table in tables:
        newTable = pd.read_html(table.get_attribute('outerHTML'))
        if newTable:
            daily = newTable[0].fillna('')
            daily["date"] = date
    data_store = pd.concat([data_store,daily])
    driver.quit()
data_store.to_csv("weather.csv")

In [5]:
# transfer 12 hours AM/PM to 24 hours
def convert24(date):
    # Checking if last two elements of time 
    # is AM and first two elements are 12 
    if date[-2:] == "AM" and date[:2] == "12": 
        return "00" + ":"+ date.split()[0].split(":")[1]
          
    # remove the AM     
    elif date[-2:] == "AM": 
        return date[:-3] 
      
    # Checking if last two elements of time 
    # is PM and first two elements are 12    
    elif date[-2:] == "PM" and date[:2] == "12": 
        return date[:-3] 
          
    else: 
        # add 12 to hours and remove PM 
        return str(int(date.split()[0].split(":")[0]) + 12) + ":"+ date.split()[0].split(":")[1]

# add 9 mins     
def add_9_mins(date):

    if (int(date.split(":")[0]) + 1) == 24:
        return "00:00:00"

    else:
       return str(int(date.split(":")[0]) + 1) + ":00:00"

In [17]:
weather = pd.read_csv("weather.csv",usecols=["date","Time","Temperature","Dew Point","Humidity","Wind Speed","Condition"])
weather.drop_duplicates(subset=["Time","date"],inplace=True)

# drop the row whose frequency is over range
date_range = list(pd.bdate_range('20151231', '20201130',freq='1D').strftime("%Y-%m-%d"))
freq = list(weather[weather["date"]=="2020-01-01"]["Time"].values)
data_store = pd.DataFrame()
for i in date_range:
    mask = weather[weather["date"]==i]
    daily= mask[mask["Time"].isin(freq)]
    data_store = pd.concat([data_store,daily])
weather = data_store
weather.reset_index(drop=True,inplace=True)

In [19]:
# transfer xx:51:00 to (x+1):00:00, such as 1:51:00 to 2:00:00
weather["Time"] = weather["Time"].apply(lambda x:convert24(x)).apply(lambda x: add_9_mins(x))

# transfer x:00:00 to 0x:00:00,such as 2:00:00 to 02:00:00
weather["Time"] = weather["Time"].apply(lambda x: "0"+x.split(":")[0]+x[1:]if len(x.split(":")[0]) < 2 else (x))

# Since 23:51:00 add 9 min will become 00:00:00 of next day, we shift -1 to match
weather["date"] = weather["date"].shift(-1)
weather.drop(index=weather.iloc[[-1]].index,inplace =True)

# combine the date and time, which will be YYYY/MM/DD HH:MM:SS
weather["date"] = weather["date"].apply(lambda x: x.replace("-","/"))

#start from 2016
weather = weather[weather["date"].apply(lambda x: x.split("/")[0]).astype(int) > 2015]
weather.reset_index(drop=True,inplace=True)

In [20]:
# check missing values
print("Checking which feature which missive values")
display(weather.isnull().mean())
print("-"*80)

#Condition
print("The 'Conditon' has a missing values")
condition = weather["Condition"]
miss_index = int(condition[condition.isnull()].index.values)
display(weather.loc[[miss_index-1,miss_index,miss_index+1]])

# fill missing values by using previous values
print(" "*30+"↓ ↓ ↓ ↓ ↓" + " "*30)
weather.loc[miss_index,"Condition"] = weather.loc[miss_index+1,"Condition"]
display(weather.loc[[miss_index]])

Checking which feature which missive values


Time           0.000000
Temperature    0.000000
Dew Point      0.000000
Humidity       0.000000
Wind Speed     0.000000
Condition      0.000023
date           0.000000
dtype: float64

--------------------------------------------------------------------------------
The 'Conditon' has a missing values


,Time,Temperature,Dew Point,Humidity,Wind Speed,Condition,date
40128,01:00:00,80 F,70 F,71 %,9 mph,Cloudy,2020/08/02
40129,02:00:00,80 F,69 F,69 %,9 mph,NaN,2020/08/02
40130,03:00:00,79 F,70 F,74 %,7 mph,Cloudy,2020/08/02


                              ↓ ↓ ↓ ↓ ↓                              


,Time,Temperature,Dew Point,Humidity,Wind Speed,Condition,date
40129,02:00:00,80 F,69 F,69 %,9 mph,Cloudy,2020/08/02


In [22]:
# remove the windy
weather["Condition"] = pd.Series(weather["Condition"].values).apply(lambda x: x.split(" /")[0])

# narrow the cloudy class, transfer Mostly Cloudy and Partly Cloudy to Cloudy
weather.replace([weather["Condition"][pd.Series(weather["Condition"].values).apply(lambda x: "Cloudy" in x)].value_counts().index.values],"Cloudy",inplace = True)
weather.replace([weather["Condition"][pd.Series(weather["Condition"].values).apply(lambda x: "Rain" in x)].value_counts().index.values],"Rain", inplace = True)
weather.replace([weather["Condition"][pd.Series(weather["Condition"].values).apply(lambda x: "T-Storm" in x)].value_counts().index.values],"Rain", inplace = True)
weather.replace([weather["Condition"][pd.Series(weather["Condition"].values).apply(lambda x: "Drizzle" in x)].value_counts().index.values],"Rain", inplace = True)
weather.replace([weather["Condition"][pd.Series(weather["Condition"].values).apply(lambda x: "Sleet" in x)].value_counts().index.values],"Rain", inplace = True)
weather.replace([weather["Condition"][pd.Series(weather["Condition"].values).apply(lambda x: "Thunder" in x)].value_counts().index.values],"Thunder", inplace = True)
weather.replace([weather["Condition"][pd.Series(weather["Condition"].values).apply(lambda x: "Squalls" in x)].value_counts().index.values],"Thunder", inplace = True)
weather.replace([weather["Condition"][pd.Series(weather["Condition"].values).apply(lambda x: "Snow" in x)].value_counts().index.values],"Snow", inplace = True)
weather.replace([weather["Condition"][pd.Series(weather["Condition"].values).apply(lambda x: "Haze" in x)].value_counts().index.values],"Fog", 
inplace = True)
weather.replace([weather["Condition"][pd.Series(weather["Condition"].values).apply(lambda x: "Fog" in x)].value_counts().index.values],"Fog", 
inplace = True)
weather.replace([weather["Condition"][pd.Series(weather["Condition"].values).apply(lambda x: "Mist" in x)].value_counts().index.values],"Fog", 
inplace = True)
weather.replace([weather["Condition"][pd.Series(weather["Condition"].values).apply(lambda x: "Small Hail" in x)].value_counts().index.values],"Rain", inplace = True)

weather["Condition"].value_counts()

Cloudy        28022
Fair           9993
Rain           3775
Fog             647
Snow            404
Wintry Mix       77
Thunder          72
Name: Condition, dtype: int64

In [24]:
# remove unit
for i in ["Temperature","Dew Point","Humidity","Wind Speed"]:
    weather[i] = weather[i].apply(lambda x: x.split()[0])

In [26]:
# check the which date which are not 24 hours
mask = weather["date"].value_counts()
date_with_miss_time = mask[mask!=24]

print("Drop the date which miss over 9 number of hours,including: ")
drop_index = list(date_with_miss_time[date_with_miss_time<=15].index)
print(drop_index)
for i in drop_index:
    weather.drop(index= weather[weather["date"] == i].index, inplace = True)

Drop the date which miss over 9 number of hours,including: 
['2017/07/08', '2020/11/09']


In [35]:
# use data of the previous hour data to fill data of missing hours
data_store = pd.DataFrame()
for date in weather["date"].value_counts().sort_index().index:
    mask = weather[weather["date"] == date]
    full_time = pd.DataFrame(weather[weather["date"]=="2020/01/01"]["Time"].values,columns=["Time"])
    check = full_time.merge(mask,on="Time",how="left")
    check.fillna(method="ffill",inplace = True)
    data_store = pd.concat([data_store,check])
weather = data_store 

In [37]:
weather["date"] = weather["date"] + " "+ weather["Time"]
weather.drop(columns = "Time",inplace = True)

In [38]:
weather.to_csv("weather_features.csv")